In [8]:
%matplotlib widget

import sys
sys.path.append("../")

import yaml
import matplotlib.pyplot as plt
import cattrs
from typing import List
from mixture_optimization.datamodels.trial_tracking_config import Experiment, ValResult, Dict
from collections import defaultdict
import numpy as np


from law import read_experiments, parse_results, fit, optimize
from copy import deepcopy

In [9]:
# Merge both experiments
path_1 = "../logs/uniform_books_cc_stack_peso_reddit/experiment_history.yaml"
path_2 = "../logs/uniform_books_cc_stack_pes20_reddit_0/experiment_history.yaml"

In [10]:
def filter_trial(experiment: Experiment):
    experiment.trials = [trial for trial in experiment.trials if trial.weighted_val_perplexity is not None]
    return experiment

In [11]:
experiment_1 = read_experiments(path_1)[0]
experiment_2 = read_experiments(path_2)[0]

experiment_1 = filter_trial(experiment_1)
experiment_2 = filter_trial(experiment_2)

print(f"Experiment 1: {len(experiment_1.trials)} trials")
print(f"Experiment 2: {len(experiment_2.trials)} trials")

experiment_merged = deepcopy(experiment_1)
experiment_merged.trials.extend(experiment_2.trials)
print(f"Experiment merged: {len(experiment_merged.trials)} trials")

Experiment 1: 32 trials
Experiment 2: 11 trials
Experiment merged: 43 trials


In [17]:
def func(r, B, b):
    eps = 1e-6
    return B / (r ** b + eps)

best_parameters, results = fit(experiment_merged, func, region=(0.05, 0.9))
best_weights, pred_value = optimize(func, best_parameters)

domain_names = list(best_parameters.keys())
weight_str = [f"{name}: {weight}" for (name, weight) in zip(domain_names, best_weights)]
print(f"Best weights: {weight_str}, predicted value: {pred_value}")

Best weights: ['Books: 0.1665496656343505', 'CC: 0.0928666813209978', 'Pes2O: 0.22210864004797307', 'Reddit: 0.09975565527299625', 'stack-v4: 0.4187193577236824'], predicted value: 4.382244095522617
